In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# LOADING DATASET

In [2]:
train_dataset = dsets.MNIST(root='./data',train=True, transform=transforms.ToTensor(),download=True)

test_dataset = dsets.MNIST(root='./data',train=False, transform=transforms.ToTensor(),download=True)

# MAKING DATASET ITERABLE

In [3]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [4]:
print(num_epochs)

5


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# CREATE MODEL CLASS

In [6]:
class Feedforward_NN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Feedforward_NN, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()

        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # Linear function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.relu3 = nn.ReLU()

        # Linear function 4 (readout): 100 --> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        return out

# INSTANTIATE MODEL CLASS

In [7]:
input_dim = 784
hidden_dim = 100
output_dim = 10

model = Feedforward_NN(input_dim, hidden_dim, output_dim)

# INSTANTIATE LOSS & OPTIMIZER CLASS

In [8]:
criterion = nn.CrossEntropyLoss()

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# TRAIN THE MODEL

In [9]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 784).requires_grad_()
        labels = labels

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 100 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 784).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 100. Loss: 1.7863355875015259. Accuracy: 48.349998474121094
Iteration: 200. Loss: 0.686348557472229. Accuracy: 82.30000305175781
Iteration: 300. Loss: 0.5043839812278748. Accuracy: 86.37000274658203
Iteration: 400. Loss: 0.401513934135437. Accuracy: 88.95999908447266
Iteration: 500. Loss: 0.2960836887359619. Accuracy: 91.36000061035156
Iteration: 600. Loss: 0.28928419947624207. Accuracy: 90.87999725341797
Iteration: 700. Loss: 0.26141414046287537. Accuracy: 93.0
Iteration: 800. Loss: 0.2876971960067749. Accuracy: 93.26000213623047
Iteration: 900. Loss: 0.23296776413917542. Accuracy: 93.68000030517578
Iteration: 1000. Loss: 0.12800857424736023. Accuracy: 94.06999969482422
Iteration: 1100. Loss: 0.19017302989959717. Accuracy: 94.4000015258789
Iteration: 1200. Loss: 0.1466958075761795. Accuracy: 94.87999725341797
Iteration: 1300. Loss: 0.15967686474323273. Accuracy: 95.0999984741211
Iteration: 1400. Loss: 0.22566215693950653. Accuracy: 94.87000274658203
Iteration: 1500. Loss: 0